In [1]:
import numpy as np

from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Estimator
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp

import time

In [2]:
num_layers = 5
n_qubits = 3

inputs = ParameterVector("input", n_qubits)
weights = ParameterVector("weights", num_layers * n_qubits)

qc = QuantumCircuit(3)
for i in range(n_qubits):
    qc.rx(inputs[i], i)

for i in range(n_qubits):
    qc.ry(weights[i], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rz(weights[i+3], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rx(weights[i+6], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rz(weights[i+9], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rz(weights[i+12], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

In [3]:
qc.draw()

┌──────────────┐┌────────────────┐          ┌───┐┌────────────────┐     »
q_0: ┤ Rx(input[0]) ├┤ Ry(weights[0]) ├──■───────┤ X ├┤ Rz(weights[3]) ├──■──»
     ├──────────────┤├────────────────┤┌─┴─┐     └─┬─┘├────────────────┤┌─┴─┐»
q_1: ┤ Rx(input[1]) ├┤ Ry(weights[1]) ├┤ X ├──■────┼──┤ Rz(weights[4]) ├┤ X ├»
     ├──────────────┤├────────────────┤└───┘┌─┴─┐  │  ├────────────────┤└───┘»
q_2: ┤ Rx(input[2]) ├┤ Ry(weights[2]) ├─────┤ X ├──■──┤ Rz(weights[5]) ├─────»
     └──────────────┘└────────────────┘     └───┘     └────────────────┘     »
«          ┌───┐┌────────────────┐          ┌───┐ ┌────────────────┐          »
«q_0: ─────┤ X ├┤ Rx(weights[6]) ├──■───────┤ X ├─┤ Rz(weights[9]) ├──■───────»
«          └─┬─┘├────────────────┤┌─┴─┐     └─┬─┘┌┴────────────────┤┌─┴─┐     »
«q_1: ──■────┼──┤ Rx(weights[7]) ├┤ X ├──■────┼──┤ Rz(weights[10]) ├┤ X ├──■──»
«     ┌─┴─┐  │  ├────────────────┤└───┘┌─┴─┐  │  ├─────────────────┤└───┘┌─┴─┐»
«q_2: ┤ X ├──■──┤ Rx(weights[8]) ├─────┤ X ├──■──┤ Rz(weights[11]) ├─────┤ X ├»
«     └───┘     └────────────────┘     └───┘     └─────────────────┘     └───┘»
«     ┌───┐┌─────────────────┐          ┌───┐
«q_0: ┤ X ├┤ Rz(weights[12]) ├──■───────┤ X ├
«     └─┬─┘├─────────────────┤┌─┴─┐     └─┬─┘
«q_1: ──┼──┤ Rz(weights[13]) ├┤ X ├──■────┼──
«       │  ├─────────────────┤└───┘┌─┴─┐  │  
«q_2: ──■──┤ Rz(weights[14]) ├─────┤ X ├──■──
«          └─────────────────┘     └───┘

In [4]:
obs = SparsePauliOp("ZZI")

In [5]:
service = QiskitRuntimeService()
backend = service.get_backend("ibmq_qasm_simulator")

In [6]:
weights_ = np.random.randn(num_layers * n_qubits)
inputs_ = np.random.randn(n_qubits)

In [7]:
start = time.time()
now = start

with Session(service=service, backend=backend) as session:
    estimator = Estimator(session=session, backend=backend)

    for i in range(5):
        qc_ = qc.assign_parameters({weights:weights_, inputs:inputs_})
        job = estimator.run(qc_, obs, shots=1024)
        result = job.result()
        print(result.values)

        
        weights_ += 0.2* np.random.randn(num_layers * n_qubits)
        inputs_ += 0.2*np.random.randn(n_qubits)

        now, last = time.time(), now
        print(now-last)

[-0.78320312]
4.165259838104248
[-0.69140625]
3.879397392272949
[-0.69726562]
2.8849000930786133
[-0.34375]
2.6587846279144287
[0.0234375]
4.3773534297943115


In [8]:
print(f"Total running cost {now-start:.1f} seconds")

Total running cost 18.0 seconds
